![image](car.jpeg)

**Car-ing is sharing**, an auto dealership company for car sales and rental, is taking their services to the next level thanks to **Large Language Models (LLMs)**.

As their newly recruited AI and NLP developer, you've been asked to prototype a chatbot app with multiple functionalities that not only assist customers but also provide support to human agents in the company.

The solution should receive textual prompts and use a variety of pre-trained Hugging Face LLMs to respond to a series of tasks, e.g. classifying the sentiment in a car’s text review, answering a customer question, summarizing or translating text, etc.


In [18]:
# Import necessary packages
import pandas as pd
import torch

from transformers import logging
logging.set_verbosity(logging.WARNING)

In [19]:
car_reviews = pd.read_csv('car_reviews.csv', sep=';')

print(car_reviews.shape)
car_reviews.head()

(5, 2)


,Review,Class
0,I am very satisfied with my 2014 Nissan NV SL....,POSITIVE
1,The car is fine. It's a bit loud and not very ...,NEGATIVE
2,"My first foreign car. Love it, I would buy ano...",POSITIVE
3,I've come across numerous reviews praising the...,NEGATIVE
4,I've been dreaming of owning an SUV for quite ...,POSITIVE


In [20]:
car_reviews.Review.iloc[0]

'I am very satisfied with my 2014 Nissan NV SL. I use this van for my business deliveries and personal use. Camping, road trips, etc. We dont have any children so I store most of the seats in my warehouse. I wanted the passenger van for the rear air conditioning. We drove our van from Florida to California for a Cross Country trip in 2014. We averaged about 18 mpg. We drove thru a lot of rain and It was a very comfortable and stable vehicle. The V8 Nissan Titan engine is a 500k mile engine. It has been tested many times by delivery and trucking companies. This is why Nissan gives you a 5 year or 100k mile bumper to bumper warranty. Many people are scared about driving this van because of its size. But with front and rear sonar sensors, large mirrors and the back up camera. It is easy to drive. The front and rear sensors also monitor the front and rear sides of the bumpers making it easier to park close to objects. Our Nissan NV is a Tow Monster. It pulls our 5000 pound travel trailer l

In [21]:
from transformers import pipeline

# Load a lightweight sentiment analysis pipeline
sentiment_analyzer = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")

# Analyze sentiment for all car reviews
results = sentiment_analyzer(list(car_reviews.Review))
print(results)

[{'label': 'POSITIVE', 'score': 0.929397702217102}, {'label': 'POSITIVE', 'score': 0.8654271364212036}, {'label': 'POSITIVE', 'score': 0.9994640946388245}, {'label': 'NEGATIVE', 'score': 0.9935314059257507}, {'label': 'POSITIVE', 'score': 0.9986565113067627}]


In [22]:
import evaluate

accuracy = evaluate.load('accuracy')
# Store the model outputs in predicted_labels
predicted_labels = [res['label'] for res in results]

# Map 'POSITIVE' to 1 and 'NEGATIVE' to 0
label_map = {'POSITIVE': 1, 'NEGATIVE': 0}
predictions = [label_map[label] for label in predicted_labels]

# Extract ground truth labels and map to binary
references = [label_map[label] for label in car_reviews['Class']]

# Evaluate accuracy and F1 score
accuracy_result = accuracy.compute(predictions=predictions, references=references)
f1 = evaluate.load('f1')
f1_result = f1.compute(predictions=predictions, references=references, average='binary')

print("Accuracy:", accuracy_result)
print("F1 Score:", f1_result)

Accuracy: {'accuracy': 0.8}
F1 Score: {'f1': 0.8571428571428571}


In [23]:
%pip install sentencepiece

from transformers import pipeline

# Load a lightweight English-to-Spanish translation pipeline
translator = pipeline("translation_en_to_es", model="Helsinki-NLP/opus-mt-en-es")


Note: you may need to restart the kernel to use updated packages.


c:\Users\El-Wattaneya\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\models\marian\tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [24]:
import re

text = car_reviews.Review.iloc[0]
sentences = re.split(r'(?<=[.!?])\s+', text)
first_two = ' '.join(sentences[:2])
print(first_two)

I am very satisfied with my 2014 Nissan NV SL. I use this van for my business deliveries and personal use.


In [25]:
print(translator(first_two))
translated_review = translator(first_two)[0]['translation_text']
translated_review

[{'translation_text': 'Estoy muy satisfecho con mi Nissan NV SL 2014. Uso esta camioneta para mis entregas de negocios y uso personal.'}]


'Estoy muy satisfecho con mi Nissan NV SL 2014. Uso esta camioneta para mis entregas de negocios y uso personal.'

In [26]:
with open('reference_translations.txt', 'r', encoding='utf-8') as f:
	reference_translations = [line.strip() for line in f if line.strip()]
	
reference_translations[:]  

['Estoy muy satisfecho con mi Nissan NV SL 2014. Utilizo esta camioneta para mis entregas comerciales y uso personal.',
 'Estoy muy satisfecho con mi Nissan NV SL 2014. Uso esta furgoneta para mis entregas comerciales y uso personal.']

In [27]:
BLEU = evaluate.load('bleu')

BLEU.features

[{'predictions': Value(dtype='string', id='sequence'),
  'references': Sequence(feature=Value(dtype='string', id='sequence'), length=-1, id='references')},
 {'predictions': Value(dtype='string', id='sequence'),
  'references': Value(dtype='string', id='sequence')}]

In [28]:
bleu_score= BLEU.compute(predictions=[translated_review], references=[reference_translations])
bleu_score

{'bleu': 0.7794483794144498,
 'precisions': [0.9090909090909091,
  0.8571428571428571,
  0.75,
  0.631578947368421],
 'brevity_penalty': 1.0,
 'length_ratio': 1.0476190476190477,
 'translation_length': 22,
 'reference_length': 21}